<a href="https://colab.research.google.com/github/blazyy/Coursera_Capstone/blob/master/Clustering_Neighborhoods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing and Preprocessing

In [0]:
import pandas as pd
import numpy as np

Reading in dataframe using read_html() function in pandas.

In [0]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

Removing boroughs that are unassigned.

In [35]:
df = df[df['Borough'] != 'Not assigned']
df.head()

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


Changing the neighborhood to be the same as boroughs if they are not assigned.

In [0]:
df.Neighborhood = df.Borough.where(df.Neighborhood == 'Not assigned', df.Neighborhood)

Aggregating rows so that only one row exists per postcode, and then converting the lists into a string of neighborhoods separated by commas.

In [37]:
df = df['Neighborhood'].groupby([df.Postcode, df.Borough]).apply(list).reset_index()
df['Neighborhood'] = df['Neighborhood'].apply(lambda x: ', '.join(x))
df

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


In [38]:
print(df.shape)

(103, 3)


# Loading coordinates and combining dataframes

In [39]:
!wget https://cocl.us/Geospatial_data

--2020-01-13 05:48:55--  https://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 159.8.69.21, 159.8.69.24, 159.8.72.228
Connecting to cocl.us (cocl.us)|159.8.69.21|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-01-13 05:48:58--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 185.235.236.197
Connecting to ibm.box.com (ibm.box.com)|185.235.236.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-01-13 05:48:58--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Reusing existing connection to ibm.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [f

Loading downloaded .csv into a dataframe

In [0]:
coords_df = pd.read_csv('Geospatial_data')

Combining both dataframes using an inner join on the postal code columns.

In [41]:
neighborhoods = pd.merge(left = df, right = coords_df, left_on = 'Postcode', right_on = 'Postal Code')
neighborhoods = neighborhoods.drop('Postcode', axis = 1)
neighborhoods.head()

,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,Scarborough,Woburn,M1G,43.770992,-79.216917
4,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


# Analyzing Neighborhoods

In [0]:
import folium
import requests

Using only the neighborhoods that contain the word 'Toronto' for simplification.

In [0]:
toronto_neighborhoods = neighborhoods[neighborhoods['Borough'].str.contains('Toronto')]

In [44]:
# Creating a map of Toronto, Canada

map_toronto = folium.Map(location=[43.6532, -79.3832], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_neighborhoods['Latitude'], toronto_neighborhoods['Longitude'], toronto_neighborhoods['Borough'], toronto_neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [45]:
CLIENT_ID = '0IFY4PNY5WFYARBFVL5QOSL2OGF1RY4MBS45UTK042LXJYEB' # your Foursquare ID
CLIENT_SECRET = 'FX0Z0B54LNCTOSEZDWVMDNS3CWT4P3TTSBU2JMZMGSK1GKT4' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0IFY4PNY5WFYARBFVL5QOSL2OGF1RY4MBS45UTK042LXJYEB
CLIENT_SECRET:FX0Z0B54LNCTOSEZDWVMDNS3CWT4P3TTSBU2JMZMGSK1GKT4


A function to get nearby venues.

In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Getting all venus in the Toronto neighborhoods.

In [47]:
toronto_venues = getNearbyVenues(names = toronto_neighborhoods['Neighborhood'],
                                   latitudes = toronto_neighborhoods['Latitude'],
                                   longitudes = toronto_neighborhoods['Longitude']
                                  )
toronto_venues.head()

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The Junction Sout

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


Aggregating and counting the venues in each neighborhood.

In [52]:
toronto_venues.groupby('Neighborhood').count().reset_index().head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Adelaide, King, Richmond",30,30,30,30,30,30
1,Berczy Park,30,30,30,30,30,30
2,"Brockton, Exhibition Place, Parkdale Village",22,22,22,22,22,22
3,Business Reply Mail Processing Centre 969 Eastern,16,16,16,16,16,16
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",16,16,16,16,16,16


One hot encoding the neighborhoods.

In [49]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,...,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [53]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,...,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,"Adelaide, King, Richmond",0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.033333,0.0,0.000000,0.0,0.066667,0.0,0.0000,0.0,0.0,0.0,0.000000,0.0,0.033333,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0000,0.0,0.0000,0.033333,0.0000,0.0,0.0,0.000000,0.0,0.0,0.033333,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0000,0.000000,0.0,0.0,0.0,0.0,0.0000,0.033333,0.0000,0.0,0.033333,0.0,0.0000,0.033333,0.0,0.0,0.000000,0.0,0.100000,0.0,0.033333,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.0
1,Berczy Park,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,0.033333,0.0,0.000000,0.0,0.0000,0.0,0.0,0.0,0.066667,0.0,0.000000,0.033333,0.066667,0.0,0.0,0.033333,0.0,0.0000,0.0,0.0000,0.033333,0.0000,0.0,0.0,0.000000,0.0,0.0,0.066667,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0000,0.000000,0.0,0.0,0.0,0.0,0.0000,0.066667,0.0000,0.0,0.000000,0.0,0.0000,0.000000,0.0,0.0,0.000000,0.0,0.033333,0.0,0.000000,0.0,0.0,0.0,0.0,0.033333,0.033333,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.0
2,"Brockton, Exhibition Place, Parkdale Village",0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0000,0.0,0.0,0.0,0.045455,0.0,0.045455,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0000,0.0,0.0000,0.090909,0.0000,0.0,0.0,0.045455,0.0,0.0,0.136364,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0000,0.045455,0.0,0.0,0.0,0.0,0.0000,0.000000,0.0000,0.0,0.000000,0.0,0.0000,0.000000,0.0,0.0,0.045455,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,Business Reply Mail Processing Centre 969 Eastern,0.0625,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0625,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0000,0.0,0.0000,0.000000,0.0625,0.0,0.0,0.062500,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0000,0.062500,0.0,0.0,0.0,0.0,0.0000,0.000000,0.0625,0.0,0.062500,0.0,0.0625,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.0000,0.0625,0.0625,0.125,0.1875,0.125,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0000,0.0,0.0,0.0,0.000000,0.0,0.062500,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0625,0.0,0.0625,0.000000,0.0000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0625,0.000000,0.0,0.0,0.0,0.0,0.0625,0.000000,0.0000,0.0,0.000000,0.0,0.0000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


Each neighborhood along with the top 10 most frequent venues.

In [51]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Steakhouse,Coffee Shop,Hotel,Asian Restaurant,Gym / Fitness Center,Breakfast Spot,Plaza,Café,Monument / Landmark,Smoke Shop
1,Berczy Park,Coffee Shop,Cocktail Bar,Café,Seafood Restaurant,Farmers Market,Beer Bar,Bakery,Park,Steakhouse,Concert Hall
2,"Brockton, Exhibition Place, Parkdale Village",Café,Breakfast Spot,Coffee Shop,Nightclub,Pet Store,Bar,Bakery,Intersection,Italian Restaurant,Climbing Gym
3,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Spa,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Light Rail Station,Comic Shop,Pizza Place,Burrito Place
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Harbor / Marina,Boutique,Rental Car Location,Bar,Plane,Sculpture Garden


# Clustering

In [0]:
import matplotlib.cm as cm
import matplotlib.colors as colors

Setting up kmeans.

In [0]:
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_[0:10] 

A new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [56]:
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = toronto_neighborhoods
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() 

,Borough,Neighborhood,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,The Beaches,M4E,43.676357,-79.293031,0,Other Great Outdoors,Trail,Health Food Store,Pub,Comfort Food Restaurant,Creperie,Dumpling Restaurant,Donut Shop,Dog Run,Diner
41,East Toronto,"The Danforth West, Riverdale",M4K,43.679557,-79.352188,0,Greek Restaurant,Italian Restaurant,Ice Cream Shop,Yoga Studio,Pizza Place,Juice Bar,Bookstore,Restaurant,Spa,Diner
42,East Toronto,"The Beaches West, India Bazaar",M4L,43.668999,-79.315572,0,Park,Board Shop,Liquor Store,Brewery,Burger Joint,Burrito Place,Italian Restaurant,Intersection,Ice Cream Shop,Pub
43,East Toronto,Studio District,M4M,43.659526,-79.340923,0,Café,Coffee Shop,Italian Restaurant,American Restaurant,Bakery,Sandwich Place,Fish Market,Ice Cream Shop,Seafood Restaurant,Bookstore
44,Central Toronto,Lawrence Park,M4N,43.728020,-79.388790,2,Park,Bus Line,Construction & Landscaping,Swim School,Ethiopian Restaurant,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run,Diner


In [60]:
map_clusters = folium.Map(location=[43.6532, -79.3832], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examining Clusters

In [67]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,The Beaches,0,Other Great Outdoors,Trail,Health Food Store,Pub,Comfort Food Restaurant,Creperie,Dumpling Restaurant,Donut Shop,Dog Run,Diner
41,"The Danforth West, Riverdale",0,Greek Restaurant,Italian Restaurant,Ice Cream Shop,Yoga Studio,Pizza Place,Juice Bar,Bookstore,Restaurant,Spa,Diner
42,"The Beaches West, India Bazaar",0,Park,Board Shop,Liquor Store,Brewery,Burger Joint,Burrito Place,Italian Restaurant,Intersection,Ice Cream Shop,Pub
43,Studio District,0,Café,Coffee Shop,Italian Restaurant,American Restaurant,Bakery,Sandwich Place,Fish Market,Ice Cream Shop,Seafood Restaurant,Bookstore
45,Davisville North,0,Park,Food & Drink Shop,Breakfast Spot,Sandwich Place,Dog Run,Dance Studio,Hotel,Gym,Clothing Store,Concert Hall
46,North Toronto West,0,Sporting Goods Shop,Coffee Shop,Clothing Store,Yoga Studio,Gym / Fitness Center,Diner,Dessert Shop,Mexican Restaurant,Chinese Restaurant,Café
47,Davisville,0,Dessert Shop,Italian Restaurant,Café,Coffee Shop,Gym,Sandwich Place,Sushi Restaurant,Pizza Place,Costume Shop,Farmers Market
49,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",0,Coffee Shop,Pub,Supermarket,Sushi Restaurant,Sports Bar,Pizza Place,Athletics & Sports,Liquor Store,Light Rail Station,Fried Chicken Joint
51,"Cabbagetown, St. James Town",0,Restaurant,Italian Restaurant,Coffee Shop,Café,Indian Restaurant,Diner,Sandwich Place,Jewelry Store,Pub,Butcher
52,Church and Wellesley,0,Burger Joint,Gay Bar,Wine Shop,Pizza Place,Hobby Shop,Juice Bar,Beer Bar,Ethiopian Restaurant,Diner,Pub


In [68]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,"Moore Park, Summerhill East",1,Playground,Wine Shop,Cuban Restaurant,Ethiopian Restaurant,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run,Diner,Dessert Shop


In [69]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Lawrence Park,2,Park,Bus Line,Construction & Landscaping,Swim School,Ethiopian Restaurant,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run,Diner
50,Rosedale,2,Park,Trail,Playground,Creperie,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run,Diner,Dessert Shop


In [63]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
64,"Forest Hill North, Forest Hill West",3,Jewelry Store,Trail,Sushi Restaurant,Mexican Restaurant,Dance Studio,Ethiopian Restaurant,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run


In [65]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Roselawn,4,Garden,Wine Shop,Cuban Restaurant,Ethiopian Restaurant,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run,Diner,Dessert Shop


* According to the above information, neighborhoods in cluster 0 all have various food related venues near them. Coffee shops, restaurants, grocery stores etc. 

* Clusters 1, 2, 3, 4 don't have a lot of neighborhoods in them. This suggests that they are unique neighborhoods which cannot be classified into any big cluster.

* Cluster 1 has a lot of playgrounds.

* Cluster 2 has a lot of parks. Looking at the map, we can see that indeed neighborhoods in cluster 2 (marked by a purple dot) have a lot of green areas around.

* Cluster 3 has the most jewelry stores.

* Cluster 4 has the most gardens.

